# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value
## data preprocessing:
- stopword ✓
- lemmetize ✓
- porterstemmer (clean liked like liking into like) ✓
- embedding ✓
- train test split ✓
- word2vec after cleaning ✓

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import re,string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/aipen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aipen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aipen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aipen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df=pd.read_csv('data_references/yelp.csv')
df=df[['text','stars']]
df.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [3]:
def clean_data(word):
    model= stopwords.words('english')
    trim_s=WordNetLemmatizer()
    stem=PorterStemmer()
    word = ''.join([i for i in str(word) if not i.isdigit()])
    word = word_tokenize(word.lower())
    word = [i for i in set(word) if i.isalpha() and i not in string.punctuation and i not in model]
    word = [trim_s.lemmatize(i) for i in word]
    word = [stem.stem(i) for i in word]
    
    return " ".join(word)

In [4]:
df['true_review']=np.vectorize(clean_data)(df.text)

In [5]:
df.head()

,text,stars,true_review
0,My wife took me here on my birthday for breakf...,5,amaz morn ingredi saturday look griddl sit out...
1,I have no idea why some people give bad review...,5,peopl someon awesom everyon bad someth decid s...
2,love the gyro plate. Rice is so good and I als...,4,good also gyro plate dig select rice love candi
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5,trash locat fenc ballpark conveni let area ros...
4,General Manager Scott Petello is a good egg!!!...,5,life scott awesom import alway speak state wal...


In [6]:
X=df['true_review']
y=df.stars

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=1, window=10)

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

2022-06-10 00:29:46.566358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 00:29:46.566406: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
y_train.value_counts()

4    2347
5    2232
3     979
2     636
1     506
Name: stars, dtype: int64

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model
model=init_model()

2022-06-10 00:29:53.148563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-10 00:29:53.148977: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 00:29:53.149044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-10 00:29:53.149077: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-10 00:29:53.149134: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 64,
          epochs=10,
          callbacks=[es],
          validation_data=(X_test_pad,y_test)
         )

Epoch 1/10


ValueError: in user code:

    File "/home/aipen/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/aipen/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/aipen/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/aipen/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/home/aipen/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/home/ai

In [ ]:

!git add NLP.ipynb
!git commit -m 'built a base model, need to adjust accordingly'
!git push origin master
